# Реализация наивного Байесовского классификатора

In [1]:
import pandas as pd

filename = 'data/sms_spam_collection.tar.gz'

df = pd.read_csv(
    filename,
    compression='gzip',
    header=1,
    sep='\t',
    encoding='utf8',
    names=['class', 'sms_text'],
    error_bad_lines=False
)


df.head()

,class,sms_text
0,spam,Free entry in 2 a wkly comp to win FA Cup fina...
1,ham,U dun say so early hor... U c already then say...
2,ham,"Nah I don't think he goes to usf, he lives aro..."
3,spam,FreeMsg Hey there darling it's been 3 week's n...
4,ham,Even my brother is not like to speak with me. ...


In [2]:
df.tail()

,class,sms_text
5566,ham,Will ü b going to esplanade fr home?
5567,ham,"Pity, * was in mood for that. So...any other s..."
5568,ham,The guy did some bitching but I acted like i'd...
5569,ham,Rofl. Its true to its name
5570,NaN,NaN


In [3]:
df = df.dropna(how='all')

df.tail()

,class,sms_text
5565,spam,This is the 2nd time we have tried 2 contact u...
5566,ham,Will ü b going to esplanade fr home?
5567,ham,"Pity, * was in mood for that. So...any other s..."
5568,ham,The guy did some bitching but I acted like i'd...
5569,ham,Rofl. Its true to its name


Проверяем, сколько у нас всего объектов в датасете

In [4]:
num_objects, num_features = df.shape
print(num_objects, num_features)

5570 2


Целевая переменная (target) в столбце `class`

In [5]:
df['class'].head()

0    spam
1     ham
2     ham
3    spam
4     ham
Name: class, dtype: object

Демонстрация того, как получить булеву маску для датафрейма

In [6]:
SPAM_CLASS = 'spam'
NOT_SPAM_CLASS = 'ham'

df['class'] == SPAM_CLASS

0        True
1       False
2       False
3        True
4       False
        ...  
5565     True
5566    False
5567    False
5568    False
5569    False
Name: class, Length: 5570, dtype: bool

Использование булевой маски для фильтрации датафрейма 

In [7]:
spam_sms_num = (df['class'] == SPAM_CLASS).sum()
notspam_sms_num = (df['class'] == NOT_SPAM_CLASS).sum()

print(f'spam sms: {spam_sms_num}, not spam sms {notspam_sms_num}')

spam sms: 747, not spam sms 4823


##### Задача

считаем вероятности классов

In [8]:
# априорная вероятность класса спам
p_spam = spam_sms_num / (spam_sms_num + notspam_sms_num)

# априорная вероятность класса не спам
p_notspam = notspam_sms_num / (spam_sms_num + notspam_sms_num)

print(f'{p_spam:.4f}, {p_notspam:.4f}')

0.1341, 0.8659


Пример обработки текстовой информации - приводим к нижнему регистру

In [9]:
test_word = 'Free'.lower()

test_word

'free'

In [10]:
sms_example = df['sms_text'].values[0]

sms_example

"Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

Пишем полезные сниппеты для трансформации текста

In [11]:
# удаляем знаки препинания и цифры
import string

print(string.punctuation)
print(string.digits)

sms_example = ''.join([
    char 
    for char in sms_example 
    if (
        char not in string.punctuation 
        and 
        char not in string.digits
    )
])

sms_example

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
0123456789


'Free entry in  a wkly comp to win FA Cup final tkts st May  Text FA to  to receive entry questionstd txt rateTCs apply overs'

In [12]:
# приводим слова к нижнему регистру

sms_example = ' '.join([
    word.lower()
    for word in sms_example.split()
])

sms_example

'free entry in a wkly comp to win fa cup final tkts st may text fa to to receive entry questionstd txt ratetcs apply overs'

Объединяем сниппеты в функцию

In [13]:
import string

def text_preprocess(sms_text: str) -> str:
    """Преобразование текста для анализа"""
    text_no_punctuation = ''.join([
        char 
        for char in sms_text 
        if (
            char not in string.punctuation 
            and 
            char not in string.digits
        )
    ])
    text_lowercase = ' '.join([
        word.lower()
        for word in text_no_punctuation.split()
    ])
    
    return text_lowercase


sms_example = df['sms_text'].values[0]

print(text_preprocess(sms_example))

free entry in a wkly comp to win fa cup final tkts st may text fa to to receive entry questionstd txt ratetcs apply overs


Трансформируем каждую строчку датафрейма

In [14]:
df = df.assign(
    processed_text=df['sms_text'].apply(text_preprocess)
)

df.head()

,class,sms_text,processed_text
0,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in a wkly comp to win fa cup final ...
1,ham,U dun say so early hor... U c already then say...,u dun say so early hor u c already then say
2,ham,"Nah I don't think he goes to usf, he lives aro...",nah i dont think he goes to usf he lives aroun...
3,spam,FreeMsg Hey there darling it's been 3 week's n...,freemsg hey there darling its been weeks now a...
4,ham,Even my brother is not like to speak with me. ...,even my brother is not like to speak with me t...


In [15]:
df.tail()

,class,sms_text,processed_text
5565,spam,This is the 2nd time we have tried 2 contact u...,this is the nd time we have tried contact u u ...
5566,ham,Will ü b going to esplanade fr home?,will ü b going to esplanade fr home
5567,ham,"Pity, * was in mood for that. So...any other s...",pity was in mood for that soany other suggestions
5568,ham,The guy did some bitching but I acted like i'd...,the guy did some bitching but i acted like id ...
5569,ham,Rofl. Its true to its name,rofl its true to its name


#### Задача

Находим вероятность встретить слово в каждом из классов - это наша основная "фича" в наивном байесовском классификаторе

In [16]:
# вероятность встретить слово в спам смс
spam_messages = df[df['class'] == SPAM_CLASS]['processed_text'].tolist()
spam_test_word_entries = 0
for message in spam_messages:
    if test_word in message:
        spam_test_word_entries += 1


# вероятность встретить слово в не-спам смс
notspam_messages = df[df['class'] == NOT_SPAM_CLASS]['processed_text'].tolist()
notspam_test_word_entries = 0
for message in notspam_messages:
    if test_word in message:
        notspam_test_word_entries += 1

        
print(f'P(word="{test_word}"|class=spam)={spam_test_word_entries/spam_sms_num:.4f}')
print(f'P(word="{test_word}"|class=not_spam)={notspam_test_word_entries/notspam_sms_num:.4f}')

P(word="free"|class=spam)=0.2664
P(word="free"|class=not_spam)=0.0137


### Вывод

слово "free" встречается в спам смс с вероятностью $26.6\%$, а в не-спаме с вероятностью $1.3\%$ - т.е. это слово является хорошим "маркером" спама

# Реализовать классификатор

Аналогично тому, как посчитали вероятности встретить слово `free` в каждом классе (спам / не спам) 
* в функции `fit()` подсчитать такие вероятности для каждого слова
* в функции `predict()` по формуле байеса (см. лекцию) вычислять вероятность принадлежности входного текста к каждому из классов

Результат предсказания - класс, вероятность принадлежности к которому больше

In [17]:
from typing import List
from copy import deepcopy
from collections import defaultdict
import numpy as np

"""имплементация наивного байесовского классификатора"""
class NaiveBayes:
    def __init__(self):
        
        self.labels = [NOT_SPAM_CLASS, SPAM_CLASS]
        self.class_labels_proba = None  # априорная вероятность класса, словарь
        self.prior_word_proba = None  # частоты фичей (токенов)
        self.prediction = None  # выводить предсказания только по желанию
    
    def _set_labels_prior_proba(self, data: list, target: list) -> None:
        """Вычисление априорной вероятности классов"""
        class_labels_amount = dict.fromkeys(self.labels, 0)
        for target_type in target:
            class_labels_amount[target_type] += 1  # кол-во элементов класса
        
        all_amount = len(data)
        class_labels_proba = {}
        for target_type, target_amount in class_labels_amount.items():
            class_labels_proba.update({target_type: target_amount / all_amount})  # доля элементов класса
        
        self.class_labels_proba = class_labels_proba
    
    def _tokenize_text(self, text) -> list:
        """Функция, которая разобьёт входной текст на токены(слова)"""
        return text_preprocess(text).split()
    
    def _set_word_prior_proba(self, data, target):
        """Вычисляем априорную вероятность токенов в классе
        Заполняем словарь self.prior_word_proba[label][word]
        """
        word_dict_by_class = dict.fromkeys(self.labels)
        word_freq_by_class = dict.fromkeys(self.labels)
        word_dict_all = defaultdict(int)
        for target_type in target:
            word_dict_by_class[target_type] = defaultdict(int)
            word_freq_by_class[target_type] = defaultdict(int)

        for target_type, message in zip(target, data):  # считаем кол-во слов:
            for word in self._tokenize_text(message):
                word_dict_by_class[target_type][word] += 1  # по классу
                word_dict_all[word] += 1  # всего

        all_amount = sum(v for k, v in word_dict_all.items())  # кол-во всех слов
        for target_type in target:  # получаем долю слов в общем кол-ве
            for word in word_dict_by_class[target_type]:
                word_freq_by_class[target_type][word] = word_dict_by_class[target_type][word] / all_amount
        
        self.prior_word_proba = word_freq_by_class

    def fit(self, data: list, target: list):
        """Обучение статистик по датасету

        :param data: массив документов, каждый документ - объект типа str
        :param target: массив меток объектов
        """
        if not isinstance(data, list):
            raise ValueError('Аргумент data должен иметь тип list')
        if not isinstance(target, list):
            raise ValueError('Аргумент target должен иметь тип list')
        print('Данные инициализированы')
        self._set_labels_prior_proba(data, target)
        print(f'Априорные вероятности классов {self.class_labels_proba}')
        self._set_word_prior_proba(data, target)
        print('Обучили априорные вероятности слов')
        

    def _predict_proba(self, data: list) -> List[tuple]:
        """Предсказываем класс для текстовой смс

        :param data: массив документов, для каждого из которых нужно предсказать метку
        :return: вероятности для каждого из классов
        """
        prediction = []
        for obj in data:
            posterior_class_proba = defaultdict(lambda: 1)
            for token in self._tokenize_text(obj):
                for label in self.labels:
                    posterior_class_proba[label] *= self.prior_word_proba[label][token]
            # сохраняем для каждой метки класса - сколько меток, таков и размер tuple
            prediction.append(
                tuple(
                    posterior_class_proba[label] for label in self.labels
                )
            )
        self.prediction = prediction
        return prediction
    
    def predict(self, data) -> List[str]:
        predict_labels = []
        for proba in self._predict_proba(data):
            predict_labels.append(self.labels[np.argmax(proba)])
        return predict_labels
    

In [18]:
naive_bayes = NaiveBayes()

naive_bayes.fit(
    data=df['sms_text'].values.tolist(),
    target=df['class'].tolist()
)

Данные инициализированы
Априорные вероятности классов {'ham': 0.8658886894075404, 'spam': 0.1341113105924596}
Обучили априорные вероятности слов


In [24]:
proba = naive_bayes.prior_word_proba['ham']['thank'], naive_bayes.prior_word_proba['spam']['thank']
print("Слово 'thank':")
print(f'p_ham = {proba[0]:.6f} \np_spam = {proba[1]:.6f}')

Слово 'thank':
p_ham = 0.000324 
p_spam = 0.000012


In [27]:
proba = naive_bayes.prior_word_proba['ham']['you'], naive_bayes.prior_word_proba['spam']['you']
print("Слово 'you':")
print(f'p_ham = {proba[0]:.5f} \np_spam = {proba[1]:.5f}')

Слово 'you':
p_ham = 0.02210 
p_spam = 0.00344


Предсказание метки класса

In [20]:
import numpy as np
# рандомный объект датасета

random_obj_ind = np.random.randint(low=0, high=num_objects, size=3)
random_obj_list = df['sms_text'].values[random_obj_ind].tolist()
random_target = df['class'][random_obj_ind].tolist()

predicted_targets = naive_bayes.predict(random_obj_list)

for i, msg in enumerate(random_obj_list):
    print(f'\n{msg}')
    print(f'Predicted, Real: {predicted_targets[i]}, {random_target[i]}')


On the road so cant txt
Predicted, Real: ham, ham

Excellent! Are you ready to moan and scream in ecstasy?
Predicted, Real: ham, ham

Wen u miss someone, the person is definitely special for u..... But if the person is so special, why to miss them, just Keep-in-touch gdeve..
Predicted, Real: ham, ham


In [21]:
correct_predictions = 0
predictions = naive_bayes.predict(df['sms_text'].values.tolist())
real_classes = df['class'].tolist()

for num in range(len(predictions)):
    if predictions[num] == real_classes[num]:
        correct_predictions += 1
        
print(f'Доля правильных предсказаний: {correct_predictions / len(predictions)}')

Доля правильных предсказаний: 0.996588868940754


In [22]:
naive_bayes.prediction[20]

(1.0839765366399706e-33, 0.0)